### Requirments

In [1]:
!pip install pandas numpy matplotlib seaborn plotly tensorflow scikit-learn Pillow 


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip


### Imports

In [2]:
import numpy as np
import os ,PIL
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import LeakyReLU



2024-06-27 02:21:46.519865: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 02:21:46.520293: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 02:21:46.523548: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 02:21:46.563698: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 02:21:49.897332: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

### Configs

In [3]:
# Set the path to your dataset directory
dataset_dir = "crop_images"

# Set the path to the directory where you want to save the train and test data
train_dir = "Data/Train"
test_dir = "Data/Test"

valid_dir = test_dir
CLASS_LABELS  =  os.listdir(dataset_dir)
NUM_CLASSES = len(CLASS_LABELS)

EARLY_STOPPING_CRITERIA=3 
SEED = 12

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SIZE = 256

LR = 0.001
BATCH_SIZE = 10



### Train test split


In [4]:

# Set the test size (e.g., 20% of the data will be used for testing)
test_size = 0.2
train_size = 0.8
# Get the list of subdirectories (class labels) in the dataset directory
class_labels = os.listdir(dataset_dir)

# Create the train and test directories#optional
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the dataset into train and test sets for each cass
for label in class_labels:
    label_dir = os.path.join(dataset_dir, label)
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)

    # Get the list of image files in the current class directory
    images = os.listdir(label_dir)

    # Split the image files into train and test sets
    train_images, test_images = train_test_split(images, test_size=test_size,train_size=train_size)

    # Copy the train images to the train directory
    for train_image in train_images:
        src = os.path.join(label_dir, train_image)
        dst = os.path.join(train_label_dir, train_image)
        shutil.copy(src, dst)

    # Copy the test images to the test directory
    for test_image in test_images:
        src = os.path.join(label_dir, test_image)
        dst = os.path.join(test_label_dir, test_image)
        shutil.copy(src, dst)

### 3. Defining Model

In [5]:
num_classes = len(CLASS_LABELS)

# Load the pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Create a Sequential model
model = Sequential()

# Add the pre-trained MobileNet base to the Sequential model
model.add(base_model)
model.add(GlobalAveragePooling2D())

# Add multiple hidden layers
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))  # First hidden layer with LeakyReLU
model.add(Dense(64, activation='relu'))  # Second hidden layer
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))  # Third hidden layer with LeakyReLU
model.add(Dense(16, activation='relu'))
model.add(Dense(8))
model.add(LeakyReLU(alpha=0.1))  # Fourth hidden layer with LeakyReLU
model.add(Dense(4, activation='relu'))
model.add(Dense(2))
model.add(LeakyReLU(alpha=0.1))  # Fifth hidden layer with LeakyReLU 

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))  # Assuming num_classes is defined

# Make the last 20 layers of the base model trainable
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Print the model summary
model.summary()


/tmp/ipykernel_31882/2997475060.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
2024-06-27 02:22:45.479485: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-27 02:22:45.480257: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping regist

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ ?                      │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

### Model training

In [6]:
# Prepare the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.25,
    zoom_range=0.25,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20  # Adjust the number of epochs as needed
)
# Save the model
model.save('krishimitra.h5')



Found 288 images belonging to 5 classes.
Found 42 images belonging to 5 classes.
Epoch 1/20


/home/chaya/Documents/GitHub/crop-classification/cnn/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 62s 5s/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.2321 - loss: 1.6025 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.3125 - val_loss: 1.5927
Epoch 2/20


2024-06-27 02:24:03.801337: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 1.5903
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.2322 - loss: 1.5842

2024-06-27 02:24:52.952235: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Validation accuracy: {val_acc * 100:.2f}%')


# Plot training results (optional)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Generate predictions and true labels
validation_generator.reset()
Y_pred = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes[:len(y_pred)]  # Ensure lengths match

# Plot the confusion matrix
cm = confusion_matrix(y_true, y_pred)
